# Setup

In [32]:
#! pip install pandas-datareader
#! pip install pmdarima
#! pip install plotly
import numpy as np
from pandas_datareader import DataReader # pip install pandas-datareader
from pandas_datareader import data
from datetime import datetime
from pmdarima.arima import *
from pmdarima import preprocessing
from scipy import stats
from scipy.stats import skew

from statsmodels import api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

import pandas as pd
import pmdarima as pm
import plotly.graph_objects as go
import plotly.express as px
 
start = pd.to_datetime('2010-01-01') # in YYYY-MM-DD format
end = pd.to_datetime('2018-01-01')
ts = data.DataReader('NDAQ', 'yahoo', start , end) # here 'yahoo' is the API to yahoo

train = ts.Close.loc['2010-01-01':'2018-01-01']
test_7 = ts.Close.loc['2017-12-20': '2018-01-01']
test_31 = ts.Close.loc['2017-11-15':'2018-01-01']
two_month = ts.Close.loc['2017-10-15':'2018-01-01']

# Normalisation

In [33]:
# put all normalisation functions before modelling

In [34]:
#transformed data
boxCoxData, boxCox_lambda = stats.boxcox(ts.Close)
boxCoxSkew = skew(boxCoxData)

#we want to compare their absolute skewness 
if(boxCoxSkew < 0):
    boxCoxSkew = boxCoxSkew * -1

johnsonData, johnson_lambda = stats.yeojohnson(ts.Close)
johnsonSkew = skew(johnsonData)

if(johnsonSkew < 0):
    johnsonSkew = johnsonSkew * -1

#to show which is less skewed 
print(johnsonSkew)
print(boxCoxSkew)

0.043066305774378665
0.0391631256848633


In [35]:
#so we know what method was used, so we can reverse later. 
normaliseMethod = "neither"

if(johnsonSkew < boxCoxSkew):
    normalised_Data = johnsonData
    fitted_lambda = johnson_lambda
    normaliseMethod = "johnson"
    
#being == doesn't really matter same either way
elif(boxCoxSkew <= johnsonSkew):
    normalised_Data = boxCoxData
    fitted_lambda = boxCox_lambda
    normaliseMethod = "boxcox"

skew(normalised_Data)
#print(fitted_lambda)

0.0391631256848633

In [ ]:
fig = px.line(ts.Close, x=[ts.index], y="Close")
fig.show()

# Modelling Functions

## Stationarity Tests:

In [30]:
def testLevelStationarity(ts):
    adf_result = sm.tsa.stattools.adfuller(ts)
    kpps_result = sm.tsa.stattools.kpss(ts)
    print(f'ADF p-value: {adf_result[1]}')
    print(f'KPPS p-value: {kpps_result[1]}')

    d = ndiffs(ts, test='kpss')
    d += ndiffs(ts, test='adf')
    d += ndiffs(ts, test='pp')
    
    return int(d/3)

In [31]:
testLevelStationarity(ts.Close)

ADF p-value: 0.976170301796687
KPPS p-value: 0.01


C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


1

## Seasonal Tests:

In [ ]:
def seasonal_tests(data):
     
    result1 = pm.arima.nsdiffs(data, m=3, max_D=30, test='ch')
    
    print("CH results: " + str(result1))
    
    result2 = pm.arima.nsdiffs(data, m=3, max_D=30, test='ocsb')
    
    print("OCSB results: " + str(result2))
    
    return int((result1+result2)/2)


In [ ]:
seasonal_tests(normalised_Data)

## Lag Period:

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.DataFrame (diff,columns=['Difference'])

In [ ]:
def find_lag_period(data):
    
    fig = px.line(ts.Close, x=[ts.Close.index], y="Close")
    fig.add_vline(x='2010-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2011-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2012-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2013-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2014-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2015-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2016-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2017-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x='2018-01-01', line_width=3, line_dash="dash", line_color="green")
    fig.show()
    
    adf_test = ADFTest(alpha = 0.05)
    
    test = list()
    lags = [365, 182, 90, 30, 14]
    
    # original data
    test.append(adf_test.should_diff(data))
    print(test[0])
    
    # year lag
    test.append(adf_test.should_diff(difference(data, 365)))
    print(test[1])
    
    # six months lag
    test.append(adf_test.should_diff(difference(data, 182))) 
    print(test[2])
    
    # three months lag 
    test.append(adf_test.should_diff(difference(data, 90)))
    print(test[3])
    
    # one month lag 
    test.append(adf_test.should_diff(difference(data, 30)))
    print(test[4])
    
    # fortnight lag
    test.append(adf_test.should_diff(difference(data, 14))) 
    print(test[5])
    
    # finds the first lag that doesn't need to be differenced
    for i in range(6):
        if (test[i][1] == False):
            return int(lags[i])
        
    return -1

In [ ]:
print(find_lag_period(normalised_Data))
print(find_lag_period(ts.Close))

## AR & MA Tests

In [36]:
def getAicBicHqic(dataset, arimaOrder, seasonalOrder):
    
    try:
        model = sm.tsa.statespace.SARIMAX(dataset, order = arimaOrder, seasonal_order=seasonalOrder).fit(disp=False)

        aic = model.aic
        bic = model.bic
        hqic = model.hqic
        
    
    except:
        pass
    
    return aic, bic, hqic

In [37]:
def evaluateSarimaModels(dataset, pVals, dVal, qVals, seasonalPVals, seasonalDval, seasonalQVals, m):
    
    L = []
    for p in pVals:
        for q in qVals:
            for seasonalP in seasonalPVals:
                for seasonalQ in seasonalQVals:
                    try:
                        order=(p,dVal,q)
                        seasonalOrder = (seasonalP, seasonalDval, seasonalQ, m)
                        
                        model = sm.tsa.statespace.SARIMAX(dataset, order = order, seasonal_order=seasonalOrder).fit(disp=False)

                        data = [[model, (p,dVal,q), (seasonalP, seasonalDval, seasonalQ, m), model.aic, model.bic, model.hqic, model.mse, model.test_heteroskedasticity(None)[0][0]]]
                        dd = pd.DataFrame(data = data, columns = ['model','Order', 'SeasonalOrder', 'Aic', 'Bic', 'Hqic', 'Mse', 'heteroskedasticity'])
                        L.append(dd)
                    except:
                        continue
    df = pd.concat(L, ignore_index=True)
    return df



In [38]:
df = evaluateSarimaModels(normalised_Data, [0,1,2], 1, [0,1,2], [0,1,2], 1, [0,1,2], 7)

df

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: C

C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Oisin Tong\anaconda3\lib\site-packages\statsm

,model,Order,SeasonalOrder,Aic,Bic,Hqic,Mse,heteroskedasticity
0,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 0, 7)",-12074.339527,-12068.736128,-12072.282336,0.003471,0.226358
1,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 1, 7)",-13427.089823,-13415.883024,-13422.975441,0.003402,0.224102
2,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 2, 7)",-13294.811800,-13278.001602,-13288.640227,0.003407,0.226182
3,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(1, 1, 0, 7)",-12626.420172,-12615.213374,-12622.305791,0.003437,0.237493
4,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(1, 1, 1, 7)",-13425.089433,-13408.279235,-13418.917860,0.003402,0.224111
...,...,...,...,...,...,...,...,...
76,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(1, 1, 1, 7)",-13448.688495,-13409.464699,-13434.288159,0.003401,0.232378
77,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(1, 1, 2, 7)",-13450.643834,-13405.816640,-13434.186307,0.003401,0.233239
78,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(2, 1, 0, 7)",-12865.370509,-12826.146714,-12850.970173,0.003425,0.229605
79,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(2, 1, 1, 7)",-13398.995153,-13354.167958,-13382.537626,0.003403,0.229199


In [39]:
#sorts by all thes columns
df.sort_values(by=['Aic', 'Bic', 'Hqic', 'Mse', 'heteroskedasticity'])

,model,Order,SeasonalOrder,Aic,Bic,Hqic,Mse,heteroskedasticity
28,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(1, 1, 0)","(0, 1, 1, 7)",-13452.613113,-13435.802915,-13446.441540,0.003401,0.229729
32,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(1, 1, 0)","(1, 1, 2, 7)",-13452.272518,-13424.255522,-13441.986564,0.003401,0.230103
64,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 1)","(0, 1, 1, 7)",-13451.783664,-13423.766667,-13441.497710,0.003401,0.232350
55,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 0)","(0, 1, 1, 7)",-13451.417839,-13429.004242,-13443.189076,0.003401,0.230682
10,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 1)","(0, 1, 1, 7)",-13451.370682,-13434.560484,-13445.199109,0.003402,0.228810
...,...,...,...,...,...,...,...,...
18,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 2)","(0, 1, 0, 7)",-12101.925039,-12085.114841,-12095.753466,0.003469,0.238053
27,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(1, 1, 0)","(0, 1, 0, 7)",-12101.054372,-12089.847573,-12096.939990,0.003469,0.235195
72,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(0, 1, 0, 7)",-12099.046165,-12071.029168,-12088.760210,0.003469,0.238421
9,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 1)","(0, 1, 0, 7)",-12098.382106,-12087.175307,-12094.267724,0.003469,0.233411


In [ ]:
def sarima_model (data):
#train_test_split
    level_diff = testLevelStationarity(ts.Close)
    
    stepwise_model_7 = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=level_diff, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    
    return stepwise_model_7

In [ ]:
model_7 = sarima_model(ts.Close)

In [ ]:
#reverse
#restored_Data = (normalised_future_forecast*fitted_lambda +1)**(1/fitted_lambda)

In [ ]:
#evaluation of model fit
#boxljung test
#model_df is p+q
#m is seasonal period
#change boxpierce to true if you want to also run that test
#lbVal is the Ljung-Box test statistic and pVal is its p value
def getBoxLjung(data, lags, model_df, m):
    #demean the data
    demeanedData = data.sub(data.mean())
    lbVal, pVal = statsmodels.stats.diagnostic.acorr_ljungbox(demeanedData, lags, boxpierce = False, model_df = model_df, period = m)
    
    return lbVal, pVal
    

In [ ]:
#Pass a fitted model
#returns 
#JBVal = Jarque-Bera test statistic
#JBPVal = pvalue of the test statistic
def getJarqueBera(data):
    
    JBVal, JBPVal, skewness, kurtosis = statsmodels.stats.stattools.jarque_bera(data.resid)
    
    return JBVal, JBPVal, skewness, kurtosis

## Function to sort different models:


In [40]:
import csv
import statsmodels

#input is a dataframe full of models
#I assume that there is no column that represents the skewness of model
def add_skewness(df, dataset):
    skewness = []
    #loop to get summary table of every model, then get skewness
    
    #To iterate through the df and isolate the model
    len = df.index
    for i in len:

        summary_table = df.model[i].summary()

        table_csv = summary_table.as_csv();

        skewness.append(table_csv[table_csv.find('Skew')+20 : table_csv.find('Skew')+25])


        #reader object stores all data
#         reader = csv.reader(table_csv)
        #change to finds out what cell skewness is in
#         skewness.append(table_csv[11][3])
        i+=1
    #at this point, skewness array should be full
    df['skewness'] = skewness
    return df;


In [41]:
add_skewness(df,ts.Close)
df

,model,Order,SeasonalOrder,Aic,Bic,Hqic,Mse,heteroskedasticity,skewness
0,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 0, 7)",-12074.339527,-12068.736128,-12072.282336,0.003471,0.226358,-0.09
1,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 1, 7)",-13427.089823,-13415.883024,-13422.975441,0.003402,0.224102,-0.10
2,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(0, 1, 2, 7)",-13294.811800,-13278.001602,-13288.640227,0.003407,0.226182,-0.10
3,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(1, 1, 0, 7)",-12626.420172,-12615.213374,-12622.305791,0.003437,0.237493,-0.05
4,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(0, 1, 0)","(1, 1, 1, 7)",-13425.089433,-13408.279235,-13418.917860,0.003402,0.224111,-0.10
...,...,...,...,...,...,...,...,...,...
76,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(1, 1, 1, 7)",-13448.688495,-13409.464699,-13434.288159,0.003401,0.232378,-0.23
77,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(1, 1, 2, 7)",-13450.643834,-13405.816640,-13434.186307,0.003401,0.233239,-0.23
78,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(2, 1, 0, 7)",-12865.370509,-12826.146714,-12850.970173,0.003425,0.229605,0.04
79,<statsmodels.tsa.statespace.sarimax.SARIMAXRes...,"(2, 1, 2)","(2, 1, 1, 7)",-13398.995153,-13354.167958,-13382.537626,0.003403,0.229199,-0.25


## Graphing some models:

In [ ]:
# graphing 
def graphing(data, df):
    
    fig = px.line(data, x=ts.Close.index, y=[0])
    fig.data[0].line.color = 'rgb(0, 0, 0)'
    fig.data[0].name = "Original Data"
    fig.add_scatter
    
    fig.show()
    
    for i in range(6):
        model = sm.tsa.statespace.SARIMAX(normalised_Data, order = df['Order'][i], seasonal_order=df['SeasonalOrder'][i]).fit(disp=True)
        model = model.predict()
        fig = px.line(model, x=ts.Close.index, y=[0])
        fig.data[0].line.color = 'rgb(0, 0, 0)'
        fig.add_scatter
    
        fig.show()

In [ ]:
graphing(normalised_Data, df)

In [49]:
model_i = [5,7,9]
model_i = df.shape[0]
print(model_i)

81


In [74]:
fig_7 = px.line(test_7, x=test_7.index, y=test_7)
fig_31 = px.line(test_31, x=test_31.index, y=test_31)

for x in range(model_i):
    model = df.model[x]
    pred_7 = model.get_prediction(start=len(normalised_Data)-6, end=len(normalised_Data))

    pred_31 = model.get_prediction(start=len(normalised_Data)-30, end=len(normalised_Data))

    pred_7.predicted_mean = (pred_7.predicted_mean*fitted_lambda +1)**(1/fitted_lambda)
    pred_31.predicted_mean = (pred_31.predicted_mean*fitted_lambda +1)**(1/fitted_lambda)

    print("Predicted Vals of", x, ": \n", pred_7.predicted_mean)
    #print(model.summary_frame(alpha=0.05))   //mean isn't updated to reversed vals
    #conf_int = (model.conf_int(0.5)*fitted_lambda +1)**(1/fitted_lambda)   //don't think we should be reveresing the conf_int
    conf_int = (pred_7.conf_int(0.5))
    se = (pred_7.se_mean)
    print('\n',"Conf int: \n",conf_int,'\n\n')
    print('\n',"Std Err: \n",se,'\n\n')

    fig_7.add_scatter(x=test_7.index, y=pred_7.predicted_mean,name='('+''.join(str(e) for e in df.Order[x])+')'+'('+''.join(str(e) for e in df.SeasonalOrder[x])+')', mode='lines')
    fig_31.add_scatter(x=test_31.index, y=pred_31.predicted_mean, mode='lines')



Predicted Vals of 0 : 
 [75.09690543 74.40079873 75.37361959 76.16918126 75.8096068  75.81395281
 75.75816225]

 Conf int: 
 [[2.97047772 3.01713847]
 [2.96622128 3.01288202]
 [2.97215681 3.01881754]
 [2.97694393 3.02360466]
 [2.97478762 3.02144835]
 [2.97481375 3.02147449]
 [2.97447813 3.02113886]] 



 Std Err: 
 [0.01190347 0.01190347 0.01190347 0.01190347 0.01190347 0.01190347
 0.01190347] 


Predicted Vals of 1 : 
 [75.06348623 75.53622359 76.08453097 76.14473846 76.39761218 76.80827227
 76.86836289]

 Conf int: 
 [[2.97708068 3.01012895]
 [2.97994631 3.01299459]
 [2.98324361 3.01629188]
 [2.98360404 3.01665215]
 [2.98511387 3.01816197]
 [2.98755324 3.02060135]
 [2.9879089  3.020957  ]] 



 Std Err: 
 [0.00843084 0.00843084 0.00843084 0.00843079 0.00843079 0.00843079
 0.00843079] 


Predicted Vals of 2 : 
 [75.08277796 75.36506694 75.83748058 75.95910589 76.3880664  76.62076479
 76.98963512]

 Conf int: 
 [[2.97657187 3.01087248]
 [2.97828508 3.01258569]
 [2.98113527 3.01543587]


 Conf int: 
 [[2.9778336  3.0105642 ]
 [2.97957279 3.0123034 ]
 [2.98293392 3.01566452]
 [2.98395996 3.01669044]
 [2.98526608 3.01799656]
 [2.98745327 3.02018375]
 [2.9882397  3.02097018]] 



 Std Err: 
 [0.0083498  0.0083498  0.0083498  0.00834977 0.00834977 0.00834977
 0.00834977] 


Predicted Vals of 27 : 
 [75.27206558 74.36513575 75.17203128 76.07843563 75.78937533 75.71208003
 75.63813315]

 Conf int: 
 [[2.97169839 3.01804521]
 [2.9661589  3.01250573]
 [2.97109126 3.01743809]
 [2.97655783 3.02290466]
 [2.97482289 3.02116971]
 [2.97435764 3.02070446]
 [2.97391201 3.02025884]] 



 Std Err: 
 [0.01182339 0.01182339 0.01182339 0.01182339 0.01182339 0.01182339
 0.01182339] 


Predicted Vals of 28 : 
 [75.19062091 75.50209463 76.02380032 76.13775041 76.3722185  76.78092132
 76.85745674]

 Conf int: 
 [[2.97798708 3.01076805]
 [2.97987379 3.01265476]
 [2.98301344 3.0157944 ]
 [2.9836959  3.01647668]
 [2.98509618 3.01787696]
 [2.98752492 3.02030569]
 [2.98797804 3.02075881]] 



 Std 

 0.01180503] 


Predicted Vals of 55 : 
 [75.16960852 75.47753129 76.02863048 76.14990104 76.37523973 76.78218056
 76.86867154]

 Conf int: 
 [[2.97785714 3.01064277]
 [2.979723   3.01250864]
 [2.98304005 3.01582568]
 [2.98376625 3.01655171]
 [2.98511185 3.01789731]
 [2.98753004 3.02031549]
 [2.98804205 3.0208275 ]] 



 Std Err: 
 [0.00836384 0.00836384 0.00836383 0.00836379 0.00836379 0.00836379
 0.00836379] 


Predicted Vals of 56 : 
 [75.16887045 75.47098536 76.02445685 76.15161793 76.37681099 76.777061
 76.8741204 ]

 Conf int: 
 [[2.97785624 3.01063471]
 [2.97968701 3.01246549]
 [2.98301862 3.01579709]
 [2.98378009 3.01655841]
 [2.98512479 3.0179031 ]
 [2.98750328 3.0202816 ]
 [2.98807785 3.02085617]] 



 Std Err: 
 [0.00836201 0.00836201 0.00836201 0.00836197 0.00836197 0.00836197
 0.00836197] 


Predicted Vals of 57 : 
 [75.26966441 74.68535191 75.38251199 76.01247706 75.79518881 76.61192546
 76.48448398]

 Conf int: 
 [[2.97467304 3.01504144]
 [2.97111305 3.01148145]
 [2.9753

In [75]:
fig_7.show()
fig_31.show()

## Dash App

In [56]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from pandas_datareader import DataReader # pip install pandas-datareader
from pandas_datareader import data

import pandas as pd
import pmdarima as pm
import plotly.graph_objects as go
import plotly.express as px


In [57]:
#set up app

app = dash.Dash()

In [58]:
#import datasets, for example;

start = pd.to_datetime('2010-01-01') # in YYYY-MM-DD format
end = pd.to_datetime('2018-01-01')
st = data.DataReader('NDAQ', 'yahoo', start , end) 


In [59]:
# dropdown options (currently runs through NASDAQ paramteres, should be changed to relevant parameters)

features = st.columns[1:-1]
opts = [{'label' : i, 'value' : i} for i in features]

# range slider options

st['High'] = pd.to_datetime(st.High)
dates = ['2015-02-17', '2015-05-17', '2015-08-17', '2015-11-17',
         '2016-02-17', '2016-05-17', '2016-08-17', '2016-11-17', '2017-02-17']


In [60]:
# create desired plotly figure (see https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf)

trace_1 = go.Scatter(x = st.Close, y = st['Close'],
                    name = 'Close',
                    line = dict(width = 2,
                                color = 'rgb(229, 151, 50)'))
layout = go.Layout(title = 'Time Series Plot',
                   hovermode = 'closest')
fig = go.Figure(data = [trace_1], layout = layout)

In [61]:
# create a Dash layout

app.layout = html.Div([
                # a header and a paragraph
                html.Div([
                    html.H1("Time Series Analysis on Stocks Prediction"),
                    html.P("Group 40")
                         ],
                     style = {'padding' : '50px' ,
                              'backgroundColor' : '#a12f0a'}),
                # adding a plot
                dcc.Graph(id = 'plot', figure = fig_7),
                # dropdown
                html.P([
                    html.Label("Choose a Parameter"),
                    dcc.Dropdown(id = 'opt', options = opts,
                                value = opts[0])
                        ], style = {'width': '400px',
                                    'fontSize' : '20px',
                                    'padding-left' : '100px',
                                    'display': 'inline-block'}),
                # range slider
                html.P([
                    html.Label("Time Period"),
                    dcc.RangeSlider(id = 'slider',
                                    marks = {i : dates[i] for i in range(0, 9)},
                                    min = 0,
                                    max = 8,
                                    value = [1, 7])
                        ], style = {'width' : '80%',
                                    'fontSize' : '20px',
                                    'padding-left' : '100px',
                                    'display': 'inline-block'})
                      ])



In [62]:
# add callback functions

@app.callback(Output('plot', 'figure'),
             [Input('opt', 'value'),
             Input('slider', 'value')])
def update_figure(input1, input2):
    # filtering the data
    st2 = st[(st.Date > dates[input2[0]]) & (st.Date < dates[input2[1]])]
    # updating the plot
    trace_1 = go.Scatter(x = st2.Date, y = st2['High'],
                        name = 'High',
                        line = dict(width = 2,
                                    color = 'rgb(229, 151, 50)'))
    trace_2 = go.Scatter(x = st2.Date, y = st2[input1],
                        name = input1,
                        line = dict(width = 2,
                                    color = 'rgb(106, 181, 135)'))
    fig = go.Figure(data = [trace_1, trace_2], layout = layout)
    return fig
  
# add the server clause

if __name__ == '__main__':
    app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_0m1617696817.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_0m1617696817.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_0m1617696817.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_0m1617696817.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_2m1617696817.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58:51] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_15_0m1617696817.min.js HTTP/1.1" 200 -
127.0.0.1 - - [10/Apr/2021 12:58

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\dash\dash.py", line 1

127.0.0.1 - - [10/Apr/2021 12:58:52] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [10/Apr/2021 12:58:52] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.v1_15_0m1611086576.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\dash\dash.py", line 1

...

127.0.0.1 - - [10/Apr/2021 13:02:13] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\dash\dash.py", line 1

127.0.0.1 - - [10/Apr/2021 13:02:14] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Oisin Tong\anaconda3\lib\site-packages\dash\dash.py", line 1

127.0.0.1 - - [10/Apr/2021 13:02:45] "POST /_dash-update-component HTTP/1.1" 500 -


In [ ]:
#run server

app.run_server()